In [1]:
import numpy as np
import tools
import torch
import torch.nn as nn
import torch.nn.functional as F

# Some magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## Load the Data and Calculate Vocabulary

In [2]:
Languages = {
    'en':'English', 
    'es':'Spanish', 
    'pt':'Portuguese', 
    'gl':'Galician', 
    'eu':'Basque', 
    'ca':'Catalan', 
    'fr':'French', 
    'it':'Italian', 
    'de':'German'
}

## Load the Data, Calculate Vocabulary and Perplexity

In [3]:
# Load Data
data = tools.Data()
print('Size of the vocabulary: %d characters' % len(data.vocab))

Converting Training Data...
Converting Test Data...
Converting Validation Data...


In [4]:
# Data is list of tensors (each tensor is single tweet)
print(len(data.train))
print(len(data.val))
print(len(data.test))

# print(f'\n{data.train.tweet[0].size()}')
# Tensor Dimensions (sequency length, batch size, vocab)

80175
11759
14960


In [5]:
# Data is list of tensors (each tensor is single tweet)
print(len(Data.train))
print(len(Data.val))
print(len(Data.test))

print(f'\n{Data.train[0].size()}')
# Tensor Dimensions (sequence length, batch size, vocab)

80175
11759
14960

torch.Size([115, 1, 509])


## Calculate Percent out of vocab for Training and Validation

In [6]:
num_invalid_train = 0
num_invalid_val = 0
for k in range(len(Data.train)):
    num_invalid_train += torch.sum(Data.train[k][:, :, 0]).double()

for k in range(len(Data.val)):
    num_invalid_val += torch.sum(Data.val[k][:, :, 0]).double()
    
print(f'Percent of Invalid Characters - Train: {num_invalid_train / len(Data.train) * 100 : 5.5} %')
print(f'Percent of Invalid Characters - Val:   {num_invalid_val / len(Data.val) * 100 : 5.5} %')

Percent of Invalid Characters - Train:  3.0009 %
Percent of Invalid Characters - Val:    3.8949 %


## PREPLEXITY? Implement after model?
seems helpful: https://stackoverflow.com/questions/59209086/calculate-perplexity-in-pytorch

## Model

In [7]:
class MyRNN(nn.Module):
    def __init__(self, vocab_size, lang_size=9, char_vec_size=12, lang_vec_size=2, hidden_size=50):
        
        super(MyRNN, self).__init__()
        self.vocab_size = vocab_size
        self.lang_size = lang_size
        self.char_vec_size = char_vec_size
        self.lang_vec_size = lang_vec_size
        self.hidden_size = hidden_size
        
        self.char_encoder = nn.Embedding(self.vocab_size, self.char_vec_size)
        self.lang_encoder = nn.Embedding(self.lang_size, self.lang_vec_size)
        # the current hidden size = char_vec_size
        self.gru = nn.GRU(self.char_vec_size+self.lang_vec_size, self.hidden_size, num_layers=1)
        self.linear = nn.Linear(self.hidden_size, self.char_vec_size)
        self.decoder = nn.Linear(self.char_vec_size, self.vocab_size)
        
        # This shares the encoder and decoder weights as described in lecture.
        self.decoder.weight = self.char_encoder.weight
        self.decoder.bias.data.zero_()
        
        
        weight = torch.ones(vocab_size)
        # scores over PAD is not counted
        weight[PAD] = 0
        self.sm = nn.LogSoftmax(dim=1)
        self.crit = nn.NLLLoss(weight)

    def forward(self, input, hidden=None):
        emb = pack(torch.cat((self.char_encoder(input[0]), self.lang_encoder(input[1])), -1), input[2])
        output, hidden_t = self.gru(emb, hidden)
        output = unpack(output)[0]
        output = F.tanh(self.linear(output))
        output = self.decoder(output)
        return output, hidden_t

    # Predefined loss function
    def loss(self, prediction, label, reduction='elementwise_mean'):
        prediction = prediction.view(-1, self.vocab_size)
        prediction = self.sm(prediction)
        loss_val = self.crit(prediction, label.view(-1))
        return loss_val

In [8]:
input = Data.string_to_tensor('This is a test')
hidden = torch.zeros(1, 128)

output, next_hidden = rnn(input[0], hidden)
print(output.size())

torch.Size([1, 9])


## Model (John)